In [1]:
# BIT2053 E-commerce Data Analysis Project
# Python Implementation for Data Processing and Analysis with Integrated RFM

import pandas as pd
from datetime import timedelta
import warnings

warnings.filterwarnings('ignore')

class EcommerceAnalyzer:
    def __init__(self, data_path):
        """Initialize the analyzer with dataset path"""
        self.data_path = data_path
        self.df = None
        self.processed_df = None
        self.rfm_df = None
        self.analysis_date = None

    def load_data(self):
        """Load the raw dataset"""
        print("Loading dataset...")
        try:
            self.df = pd.read_excel(self.data_path)
            print(f"Dataset loaded successfully: {self.df.shape}")
            return True
        except Exception as e:
            print(f"Error loading data: {e}")
            return False

    def explore_data(self):
        """Initial data exploration"""
        print("\n=== DATA EXPLORATION ===")
        print(f"Dataset shape: {self.df.shape}")
        print(f"\nColumn names: {list(self.df.columns)}")
        print(f"\nData types:\n{self.df.dtypes}")
        print(f"\nMissing values:\n{self.df.isnull().sum()}")
        print(f"\nFirst 5 rows:\n{self.df.head()}")

        # Basic statistics
        print(f"\nBasic statistics:\n{self.df.describe()}")

    def preprocess_data(self):
        """Clean and preprocess the dataset"""
        print("\n=== DATA PREPROCESSING ===")

        # Create a copy for processing
        self.processed_df = self.df.copy()

        print(f"Original dataset shape: {self.processed_df.shape}")

        # Step 1: Remove duplicates
        original_size = len(self.processed_df)
        self.processed_df = self.processed_df.drop_duplicates()
        print(f"Removed {original_size - len(self.processed_df)} duplicate records")

        # Step 2: Handle missing values
        print(f"Missing values before cleaning:\n{self.processed_df.isnull().sum()}")

        # Remove records with missing CustomerID and Description
        self.processed_df = self.processed_df.dropna(subset=['Customer ID', 'Description'])

        # Step 3: Remove cancelled orders (negative quantities)
        self.processed_df = self.processed_df[self.processed_df['Quantity'] > 0]
        self.processed_df = self.processed_df[self.processed_df['Price'] > 0]

        # Step 4: Convert data types
        self.processed_df['InvoiceDate'] = pd.to_datetime(self.processed_df['InvoiceDate'])
        self.processed_df['Customer ID'] = self.processed_df['Customer ID'].astype(str)

        # Step 5: Create calculated fields
        self.processed_df['TotalAmount'] = self.processed_df['Quantity'] * self.processed_df['Price']
        self.processed_df['Year'] = self.processed_df['InvoiceDate'].dt.year
        self.processed_df['Month'] = self.processed_df['InvoiceDate'].dt.month
        self.processed_df['Quarter'] = self.processed_df['InvoiceDate'].dt.quarter
        self.processed_df['DayOfWeek'] = self.processed_df['InvoiceDate'].dt.day_name()

        # Step 6: Filter for valid date range (if needed)
        self.processed_df = self.processed_df[
            (self.processed_df['InvoiceDate'] >= '2010-01-01') &
            (self.processed_df['InvoiceDate'] <= '2011-12-31')
            ]

        print(f"Final processed dataset shape: {self.processed_df.shape}")
        print("Data preprocessing completed successfully!")

        return self.processed_df

    def perform_analysis(self):
        """Perform data analysis"""
        print("\n=== DATA ANALYSIS ===")

        if self.processed_df is None:
            print("Please preprocess data first!")

        # Analysis 1: Revenue Analysis
        print("\n1. REVENUE ANALYSIS")
        total_revenue = self.processed_df['TotalAmount'].sum()
        total_orders = self.processed_df['Invoice'].nunique()
        avg_order_value = total_revenue / total_orders
        unique_customers = self.processed_df['Customer ID'].nunique()
        unique_products = self.processed_df['StockCode'].nunique()

        print(f"Total Revenue: £{total_revenue:,.2f}")
        print(f"Total Orders: {total_orders:,}")
        print(f"Average Order Value: £{avg_order_value:.2f}")
        print(f"Unique Customers: {unique_customers:,}")
        print(f"Unique Products: {unique_products:,}")

        # Analysis 2: Top Products
        print("\n2. TOP PRODUCTS BY REVENUE")
        top_products = self.processed_df.groupby(['StockCode', 'Description']).agg({
            'TotalAmount': 'sum',
            'Quantity': 'sum'
        }).reset_index().sort_values('TotalAmount', ascending=False).head(10)
        print(top_products)

        # Analysis 3: Customer Analysis
        print("\n3. CUSTOMER ANALYSIS")
        customer_stats = self.processed_df.groupby('Customer ID').agg({
            'TotalAmount': ['sum', 'count', 'mean'],
            'InvoiceDate': ['min', 'max']
        }).reset_index()

        customer_stats.columns = ['Customer_ID', 'Total_Spent', 'Order_Count',
                                  'Avg_Order_Value', 'First_Purchase', 'Last_Purchase']

        # Customer segmentation
        customer_stats['Recency'] = (customer_stats['Last_Purchase'].max() -
                                     customer_stats['Last_Purchase']).dt.days

        # Basic RFM Analysis (will be enhanced in dedicated method)
        customer_stats['R_Score'] = pd.qcut(customer_stats['Recency'], 5,
                                            labels=[5, 4, 3, 2, 1], duplicates='drop')
        customer_stats['F_Score'] = pd.qcut(customer_stats['Order_Count'].rank(method='first'), 5,
                                            labels=[1, 2, 3, 4, 5], duplicates='drop')
        customer_stats['M_Score'] = pd.qcut(customer_stats['Total_Spent'], 5,
                                            labels=[1, 2, 3, 4, 5], duplicates='drop')

        customer_stats['RFM_Score'] = (customer_stats['R_Score'].astype(str) +
                                       customer_stats['F_Score'].astype(str) +
                                       customer_stats['M_Score'].astype(str))

        print("Customer Segmentation Summary:")
        print(f"High Value Customers (RFM 555): {len(customer_stats[customer_stats['RFM_Score'] == '555'])}")
        print(f"Average Customer Lifetime Value: £{customer_stats['Total_Spent'].mean():.2f}")

        # Analysis 4: Time-based Analysis
        print("\n4. TIME-BASED ANALYSIS")
        monthly_revenue = self.processed_df.groupby(['Year', 'Month'])['TotalAmount'].sum().reset_index()
        print("Monthly Revenue Trends:")
        print(monthly_revenue.head(10))

        # Analysis 5: Geographic Analysis
        print("\n5. GEOGRAPHIC ANALYSIS")
        country_analysis = self.processed_df.groupby('Country').agg({
            'TotalAmount': 'sum',
            'Customer ID': 'nunique',
            'Invoice': 'nunique'
        }).reset_index().sort_values('TotalAmount', ascending=False)

        print("Top 10 Countries by Revenue:")
        print(country_analysis.head(10))

        return {
            'revenue_stats': {
                'total_revenue': total_revenue,
                'total_orders': total_orders,
                'avg_order_value': avg_order_value,
                'unique_customers': unique_customers,
                'unique_products': unique_products
            },
            'top_products': top_products,
            'customer_stats': customer_stats,
            'monthly_revenue': monthly_revenue,
            'country_analysis': country_analysis
        }

    def calculate_rfm_metrics(self, analysis_date=None):
        """Calculate RFM metrics for each customer"""
        print("\n=== CALCULATING RFM METRICS ===")

        if self.processed_df is None:
            print("Please preprocess data first!")
            return None

        # Set analysis date (usually the day after the last transaction)
        if analysis_date is None:
            self.analysis_date = self.processed_df['InvoiceDate'].max() + timedelta(days=1)
        else:
            self.analysis_date = pd.to_datetime(analysis_date)

        print(f"Analysis Date: {self.analysis_date}")

        # Calculate RFM metrics
        rfm = self.processed_df.groupby('Customer ID').agg({
            'InvoiceDate': lambda x: (self.analysis_date - x.max()).days,  # Recency
            'Invoice': 'nunique',  # Frequency
            'TotalAmount': 'sum'   # Monetary
        }).reset_index()

        # Rename columns
        rfm.columns = ['Customer_ID', 'Recency', 'Frequency', 'Monetary']

        # Add additional customer metrics
        customer_details = self.processed_df.groupby('Customer ID').agg({
            'InvoiceDate': ['min', 'max', 'count'],
            'TotalAmount': ['mean', 'std'],
            'Quantity': 'sum',
            'Country': 'first'
        }).reset_index()

        # Flatten column names
        customer_details.columns = ['Customer_ID', 'First_Purchase_Date', 'Last_Purchase_Date',
                                    'Total_Transactions', 'Avg_Order_Value', 'Order_Value_Std',
                                    'Total_Quantity_Purchased', 'Country']

        # Merge RFM with additional details
        self.rfm_df = rfm.merge(customer_details, on='Customer_ID', how='left')

        # Calculate customer lifetime (in days)
        self.rfm_df['Customer_Lifetime_Days'] = (
                self.rfm_df['Last_Purchase_Date'] - self.rfm_df['First_Purchase_Date']
        ).dt.days

        # Fill NaN values for single-purchase customers
        self.rfm_df['Customer_Lifetime_Days'] = self.rfm_df['Customer_Lifetime_Days'].fillna(0)
        self.rfm_df['Order_Value_Std'] = self.rfm_df['Order_Value_Std'].fillna(0)

        print(f"RFM metrics calculated for {len(self.rfm_df)} customers")
        return self.rfm_df

    def assign_rfm_scores(self):
        """Assign RFM scores (1-5) using quintiles"""
        print("Assigning RFM scores...")

        if self.rfm_df is None:
            print("Please calculate RFM metrics first!")
            return None

        # Create RFM scores using quintiles
        # Recency: Lower is better (recent customers get higher scores)
        self.rfm_df['R_Score'] = pd.qcut(self.rfm_df['Recency'], 5,
                                         labels=[5, 4, 3, 2, 1], duplicates='drop')

        # Frequency: Higher is better
        self.rfm_df['F_Score'] = pd.qcut(self.rfm_df['Frequency'].rank(method='first'), 5,
                                         labels=[1, 2, 3, 4, 5], duplicates='drop')

        # Monetary: Higher is better
        self.rfm_df['M_Score'] = pd.qcut(self.rfm_df['Monetary'], 5,
                                         labels=[1, 2, 3, 4, 5], duplicates='drop')

        # Convert to string for concatenation
        self.rfm_df['R_Score'] = self.rfm_df['R_Score'].astype(str)
        self.rfm_df['F_Score'] = self.rfm_df['F_Score'].astype(str)
        self.rfm_df['M_Score'] = self.rfm_df['M_Score'].astype(str)

        # Create combined RFM score
        self.rfm_df['RFM_Score'] = (self.rfm_df['R_Score'] +
                                    self.rfm_df['F_Score'] +
                                    self.rfm_df['M_Score'])

        # Create RFM Score as numeric for easier analysis
        self.rfm_df['RFM_Score_Numeric'] = (
                self.rfm_df['R_Score'].astype(int) * 100 +
                self.rfm_df['F_Score'].astype(int) * 10 +
                self.rfm_df['M_Score'].astype(int)
        )

        print("RFM scores assigned successfully!")
        return self.rfm_df

    def create_customer_segments(self):
        """Create meaningful customer segments based on RFM scores"""
        print("Creating customer segments...")

        if self.rfm_df is None:
            print("Please calculate RFM metrics and scores first!")
            return None

        def segment_customers(row):
            """Function to assign customer segments based on RFM scores"""
            r, f, m = int(row['R_Score']), int(row['F_Score']), int(row['M_Score'])

            # Champions: High RFM scores
            if r >= 4 and f >= 4 and m >= 4:
                return "Champions"

            # Loyal Customers: High frequency and monetary, varying recency
            elif f >= 4 and m >= 4:
                return "Loyal Customers"

            # Potential Loyalists: Recent customers with good frequency
            elif r >= 4 and f >= 3:
                return "Potential Loyalists"

            # New Customers: Recent but low frequency
            elif r >= 4 and f <= 2:
                return "New Customers"

            # Promising: Recent customers with medium frequency/monetary
            elif r >= 3 and f >= 2 and m >= 2:
                return "Promising"

            # Need Attention: Above average recency, frequency and monetary
            elif r >= 3 and f >= 3 and m >= 3:
                return "Need Attention"

            # About to Sleep: Below average recency, frequency and monetary
            elif r <= 3 and f <= 3 and m <= 3:
                return "About to Sleep"

            # At Risk: Good monetary and frequency but haven't purchased recently
            elif f >= 3 and m >= 3 and r <= 2:
                return "At Risk"

            # Cannot Lose Them: High monetary, low recency
            elif m >= 4 and r <= 2:
                return "Cannot Lose Them"

            # Hibernating: Low scores across all dimensions
            elif r <= 2 and f <= 2 and m <= 2:
                return "Hibernating"

            # Lost: Lowest recency scores
            elif r <= 2:
                return "Lost"

            else:
                return "Others"

        # Apply segmentation
        self.rfm_df['Customer_Segment'] = self.rfm_df.apply(segment_customers, axis=1)

        # Add segment priorities for business action
        segment_priority = {
            "Champions": 1,
            "Loyal Customers": 2,
            "Potential Loyalists": 3,
            "Cannot Lose Them": 4,
            "At Risk": 5,
            "Need Attention": 6,
            "Promising": 7,
            "New Customers": 8,
            "About to Sleep": 9,
            "Hibernating": 10,
            "Lost": 11,
            "Others": 12
        }

        self.rfm_df['Segment_Priority'] = self.rfm_df['Customer_Segment'].map(segment_priority)

        print("Customer segments created successfully!")
        return self.rfm_df

    def perform_rfm_analysis(self):
        """Perform complete RFM analysis"""
        print("\n" + "="*60)
        print("RFM ANALYSIS")
        print("="*60)

        # Step 1: Calculate RFM metrics
        self.calculate_rfm_metrics()

        # Step 2: Assign RFM scores
        self.assign_rfm_scores()

        # Step 3: Create customer segments
        self.create_customer_segments()

        # Step 4: Generate segment summary
        segment_summary = self.generate_rfm_segment_summary()

        # Step 5: Create BI-optimized dataset
        bi_data = self.create_rfm_bi_dataset()

        print("\n" + "="*60)
        print("RFM ANALYSIS COMPLETED SUCCESSFULLY!")
        print("="*60)
        print("Key Outputs:")
        print(f"- Customer segments: {self.rfm_df['Customer_Segment'].nunique()}")
        print(f"- Total customers analyzed: {len(self.rfm_df)}")
        print(f"- BI-ready dataset created")

        return {
            'rfm_data': self.rfm_df,
            'segment_summary': segment_summary,
            'bi_data': bi_data
        }

    def generate_rfm_segment_summary(self):
        """Generate summary statistics for each customer segment"""
        print("\n=== CUSTOMER SEGMENT SUMMARY ===")

        if self.rfm_df is None:
            print("Please perform RFM analysis first!")
            return None

        segment_summary = self.rfm_df.groupby('Customer_Segment').agg({
            'Customer_ID': 'count',
            'Recency': 'mean',
            'Frequency': 'mean',
            'Monetary': ['mean', 'sum'],
            'Avg_Order_Value': 'mean',
            'Customer_Lifetime_Days': 'mean'
        }).round(2)

        # Flatten column names
        segment_summary.columns = ['Customer_Count', 'Avg_Recency', 'Avg_Frequency',
                                   'Avg_Monetary', 'Total_Revenue', 'Avg_Order_Value',
                                   'Avg_Customer_Lifetime']

        # Calculate percentages
        segment_summary['Customer_Percentage'] = (
                segment_summary['Customer_Count'] / segment_summary['Customer_Count'].sum() * 100
        ).round(2)

        segment_summary['Revenue_Percentage'] = (
                segment_summary['Total_Revenue'] / segment_summary['Total_Revenue'].sum() * 100
        ).round(2)

        # Sort by priority
        priority_order = self.rfm_df.groupby('Customer_Segment')['Segment_Priority'].first()
        segment_summary['Priority'] = segment_summary.index.map(priority_order)
        segment_summary = segment_summary.sort_values('Priority')

        print(segment_summary)
        return segment_summary

    def create_rfm_bi_dataset(self):
        """Create BI-optimized RFM dataset"""
        print("Creating BI-optimized RFM dataset...")

        if self.rfm_df is None:
            print("Please perform RFM analysis first!")
            return None

        # Create BI-optimized dataset
        bi_export = self.rfm_df.copy()

        # Add additional calculated fields for BI
        bi_export['Revenue_Per_Day'] = bi_export['Monetary'] / (bi_export['Customer_Lifetime_Days'] + 1)
        bi_export['Purchase_Frequency_Per_Month'] = bi_export['Frequency'] / ((bi_export['Customer_Lifetime_Days'] + 1) / 30)

        # Create categorical versions for better BI visualization
        bi_export['Recency_Category'] = pd.cut(bi_export['Recency'],
                                               bins=[0, 30, 90, 180, 365, float('inf')],
                                               labels=['0-30 days', '31-90 days', '91-180 days',
                                                       '181-365 days', '365+ days'])

        bi_export['Frequency_Category'] = pd.cut(bi_export['Frequency'],
                                                 bins=[0, 1, 5, 10, 20, float('inf')],
                                                 labels=['1 order', '2-5 orders', '6-10 orders',
                                                         '11-20 orders', '20+ orders'])

        bi_export['Monetary_Category'] = pd.qcut(bi_export['Monetary'], 5,
                                                 labels=['Low Value', 'Low-Medium Value',
                                                         'Medium Value', 'High-Medium Value',
                                                         'High Value'], duplicates='drop')

        print("BI-optimized dataset created successfully!")
        return bi_export

    def export_processed_data(self, filename='data/processed_ecommerce_data.csv'):
        """Export processed data for BI tools"""
        if self.processed_df is not None:
            self.processed_df.to_csv(filename, index=False)
            print(f"Processed data exported to {filename}")
        else:
            print("No processed data to export!")

    def export_rfm_data(self, filename='data/rfm_customer_segmentation.csv'):
        """Export RFM analysis results for BI tools"""
        if self.rfm_df is not None:
            # Get BI-optimized dataset
            bi_data = self.create_rfm_bi_dataset()

            # Reorder columns for better BI presentation
            column_order = [
                'Customer_ID', 'Customer_Segment', 'Segment_Priority',
                'RFM_Score', 'RFM_Score_Numeric', 'R_Score', 'F_Score', 'M_Score',
                'Recency', 'Frequency', 'Monetary',
                'Recency_Category', 'Frequency_Category', 'Monetary_Category',
                'First_Purchase_Date', 'Last_Purchase_Date', 'Customer_Lifetime_Days',
                'Total_Transactions', 'Avg_Order_Value', 'Order_Value_Std',
                'Total_Quantity_Purchased', 'Revenue_Per_Day', 'Purchase_Frequency_Per_Month',
                'Country'
            ]

            # Filter columns that exist
            available_columns = [col for col in column_order if col in bi_data.columns]
            bi_export = bi_data[available_columns]

            # Export to CSV
            bi_export.to_csv(filename, index=False)
            print(f"RFM data exported to {filename}")
            print(f"Dataset contains {len(bi_export)} customers across {bi_export['Customer_Segment'].nunique()} segments")
        else:
            print("No RFM data to export! Please run RFM analysis first.")

    def generate_summary_report(self):
        """Generate a summary report"""
        print("\n=== SUMMARY REPORT ===")

        if self.processed_df is None:
            print("Please preprocess data first!")
            return

        # Basic analysis
        analysis_results = self.perform_analysis()

        # RFM analysis results
        rfm_results = None
        if self.rfm_df is not None:
            segment_summary = self.generate_rfm_segment_summary()
            rfm_results = {
                'total_segments': self.rfm_df['Customer_Segment'].nunique(),
                'top_segment': segment_summary.index[0] if len(segment_summary) > 0 else 'N/A',
                'champions_count': len(self.rfm_df[self.rfm_df['Customer_Segment'] == 'Champions']),
                'at_risk_count': len(self.rfm_df[self.rfm_df['Customer_Segment'] == 'At Risk'])
            }

        report = f"""
E-COMMERCE DATA ANALYSIS SUMMARY REPORT
======================================

DATASET OVERVIEW:
- Total Records: {len(self.processed_df):,}
- Date Range: {self.processed_df['InvoiceDate'].min()} to {self.processed_df['InvoiceDate'].max()}
- Countries Covered: {self.processed_df['Country'].nunique()}

BUSINESS METRICS:
- Total Revenue: £{analysis_results['revenue_stats']['total_revenue']:,.2f}
- Total Orders: {analysis_results['revenue_stats']['total_orders']:,}
- Average Order Value: £{analysis_results['revenue_stats']['avg_order_value']:.2f}
- Unique Customers: {analysis_results['revenue_stats']['unique_customers']:,}
- Unique Products: {analysis_results['revenue_stats']['unique_products']:,}

RFM ANALYSIS RESULTS:
{f'- Customer Segments: {rfm_results["total_segments"]}' if rfm_results else '- RFM Analysis: Not performed'}
{f'- Champions: {rfm_results["champions_count"]} customers' if rfm_results else ''}
{f'- At Risk: {rfm_results["at_risk_count"]} customers' if rfm_results else ''}
{f'- Top Performing Segment: {rfm_results["top_segment"]}' if rfm_results else ''}

FILES GENERATED:
- processed_ecommerce_data.csv (transaction-level data for BI tools)
{f'- rfm_customer_segmentation.csv (RFM analysis for BI tools)' if self.rfm_df is not None else ''}
        """

        print(report)

        # Save report to file
        try:
            with open('report/analysis_report.txt', 'w') as f:
                f.write(report)
            print("\nreport saved as 'analysis_report.txt' in 'report' directory")
        except:
            print(f"\nReport content:\n{report}")


# Main execution
def main():
    """Main function to run the complete analysis including RFM"""
    print("BIT2053 - E-commerce Data Analysis Project with Integrated RFM")
    print("=" * 70)

    # Initialize analyzer
    analyzer = EcommerceAnalyzer('data/raw_online_retail_II.xlsx')

    # Step 1: Load data
    if analyzer.load_data():
        # Step 2: Explore data
        analyzer.explore_data()

        # Step 3: Preprocess data
        analyzer.preprocess_data()

        # Step 4: Perform basic analysis
        analyzer.perform_analysis()

        # Step 5: Perform RFM analysis
        analyzer.perform_rfm_analysis()

        # Step 6: Export processed data for BI tools
        analyzer.export_processed_data()

        # Step 7: Export RFM data for BI tools
        analyzer.export_rfm_data()

        # Step 8: Generate a summary report
        analyzer.generate_summary_report()

        print("\n" + "=" * 70)
        print("COMPLETE ANALYSIS FINISHED SUCCESSFULLY!")
        print("=" * 70)
        print("Files generated:")
        print("- processed_ecommerce_data.csv (transaction-level data for BI)")
        print("- rfm_customer_segmentation.csv (customer segmentation for BI)")
        print("- analysis_report.txt (complete summary report)")
        print("\nReady for Google Looker Studio integration!")


if __name__ == "__main__":
    main()

BIT2053 - E-commerce Data Analysis Project with Integrated RFM
Loading dataset...
Dataset loaded successfully: (1048575, 8)

=== DATA EXPLORATION ===
Dataset shape: (1048575, 8)

Column names: ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']

Data types:
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object

Missing values:
Invoice             0
StockCode           0
Description      4372
Quantity            0
InvoiceDate         0
Price               0
Customer ID    236682
Country             0
dtype: int64

First 5 rows:
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  4894